<a href="https://colab.research.google.com/github/Divyaanshvats/finrir/blob/main/fenrir_fixed.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import requests
import time
import json
from tqdm import tqdm

In [3]:
TAGS = ["bash", "git", "tar", "grep", "venv"]
BASE_URL = "https://api.stackexchange.com/2.3"
SITE = "stackoverflow"

In [4]:
import requests
import time
import json
from tqdm import tqdm

def get_questions(tag, pagesize=50):
    url = f"{BASE_URL}/questions"
    params = {
        "order": "desc",
        "sort": "votes",
        "tagged": tag,
        "site": SITE,
        "pagesize": pagesize,
        "filter": "!9_bDDxJY5",
    }
    response = requests.get(url, params=params)
    try:
        return response.json().get("items", [])
    except json.JSONDecodeError as e:
        print("Error decoding JSON:")
        print(response.text)
        raise e

In [5]:
def get_best_answer(question_id):
    url = f"{BASE_URL}/questions/{question_id}/answers"
    params = {
        "order": "desc",
        "sort": "votes",
        "site": SITE,
        "filter": "!9_bDE(fI5",
    }
    response = requests.get(url, params=params)
    if not response.text:
        print(f"Empty response received for question ID {question_id}. Possible rate limit issue.")
        return None
    answers = response.json().get("items", [])
    if not answers:
        return None
    return answers[0]["body"]

In [6]:
from bs4 import BeautifulSoup
data = []
for tag in TAGS:
    print(f"Collecting questions for tag: {tag}")
    questions = get_questions(tag, pagesize=60)
    for q in tqdm(questions):
        if not q.get("is_answered"):
            continue
        q_text = BeautifulSoup(q["title"], "html.parser").get_text()
        a_html = get_best_answer(q["question_id"])
        if not a_html:
            continue
        a_text = BeautifulSoup(a_html, "html.parser").get_text()
        data.append({
            "question": q_text.strip(),
            "answer": a_text.strip()
        })
        time.sleep(1)

100%|██████████| 60/60 [01:22<00:00,  1.38s/it]


100%|██████████| 60/60 [01:22<00:00,  1.37s/it]


100%|██████████| 60/60 [01:19<00:00,  1.32s/it]


100%|██████████| 60/60 [01:19<00:00,  1.32s/it]


100%|██████████| 17/17 [00:10<00:00,  1.60it/s]


In [7]:
import os
os.makedirs("data", exist_ok=True)
with open("data/questions.json", "w", encoding="utf-8") as f:
    json.dump(data, f, indent=2, ensure_ascii=False)

print(f"✅ Collected {len(data)} Q&A pairs.")

✅ Collected 248 Q&A pairs.


In [8]:
!pip install transformers peft accelerate datasets evaluate bitsandbytes --quiet

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.0/84.0 kB 9.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.0/67.0 MB 13.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 78.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 54.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 49.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 14.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 8.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 48.1 MB/s eta 0:00:00


In [9]:
from transformers import AutoTokenizer, AutoModelForCausalLM
from peft import LoraConfig, get_peft_model, TaskType
base_model_id = "microsoft/phi-2"
tokenizer = AutoTokenizer.from_pretrained(base_model_id)
base_model = AutoModelForCausalLM.from_pretrained(
    base_model_id,
    device_map="auto",
    torch_dtype="auto"
)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/7.34k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/798k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.11M [00:00<?, ?B/s]

...

Fetching 2 files:   0%|          | 0/2 [00:00<?, ?it/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/5.00G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/564M [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

In [10]:
from datasets import load_dataset
import json
dataset_path = "/content/data/questions.json"
with open(dataset_path, "r", encoding="utf-8") as f:
    data = json.load(f)
from datasets import Dataset
dataset = Dataset.from_list(data)
def format_example(example):
    return {
        "text": f"### Instruction:\n{example['question']}\n\n### Response:\n{example['answer']}"
    }
dataset = dataset.map(format_example)

Map:   0%|          | 0/248 [00:00<?, ? examples/s]

In [11]:
from peft import LoraConfig, TaskType, get_peft_model

In [12]:
from transformers import AutoModelForCausalLM, AutoTokenizer
from peft import LoraConfig, TaskType, get_peft_model

base_model_id = "microsoft/phi-2"
tokenizer = AutoTokenizer.from_pretrained(base_model_id)
base_model = AutoModelForCausalLM.from_pretrained(
    base_model_id,
    device_map="auto",
    torch_dtype="auto"
)
peft_config = LoraConfig(
    r=8,
    lora_alpha=16,
    lora_dropout=0.1,
    bias="none",
    task_type=TaskType.CAUSAL_LM,
    target_modules=["q_proj", "v_proj"]
)
model = get_peft_model(base_model, peft_config)
print(type(model))

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

<class 'peft.peft_model.PeftModelForCausalLM'>


In [13]:
from transformers import DataCollatorForLanguageModeling
if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token

def tokenize(example):
    return tokenizer(example["text"], truncation=True, padding="max_length", max_length=512)
tokenized_dataset = dataset.map(tokenize, batched=True)

data_collator = DataCollatorForLanguageModeling(tokenizer=tokenizer, mlm=False)

Map:   0%|          | 0/248 [00:00<?, ? examples/s]

In [14]:
from transformers import TrainingArguments, Trainer
model.print_trainable_parameters()
training_args = TrainingArguments(
    output_dir="./phi2-lora",
    per_device_train_batch_size=2,
    gradient_accumulation_steps=4,
    num_train_epochs=1,
    logging_steps=10,
    save_strategy="epoch",
    fp16=True,
    report_to="none"
)

trainable params: 2,621,440 || all params: 2,782,305,280 || trainable%: 0.0942


In [15]:
trainer = Trainer(
    model=model,
    train_dataset=tokenized_dataset,
    args=training_args,
    data_collator=data_collator
)

trainer.train()

No label_names provided for model class `PeftModelForCausalLM`. Since `PeftModel` hides base models input arguments, if label_names is not given, label_names can't be set automatically within `Trainer`. Note that empty label_names list will be used instead.


Step,Training Loss
10,2.309200
20,2.341100
30,2.372800


TrainOutput(global_step=31, training_loss=2.3449865387332056, metrics={'train_runtime': 110.963, 'train_samples_per_second': 2.235, 'train_steps_per_second': 0.279, 'total_flos': 2019857981767680.0, 'train_loss': 2.3449865387332056, 'epoch': 1.0})

In [16]:
model.save_pretrained("phi2-lora-adapter")
tokenizer.save_pretrained("phi2-lora-adapter")

('phi2-lora-adapter/tokenizer_config.json',
 'phi2-lora-adapter/special_tokens_map.json',
 'phi2-lora-adapter/vocab.json',
 'phi2-lora-adapter/merges.txt',
 'phi2-lora-adapter/added_tokens.json',
 'phi2-lora-adapter/tokenizer.json')

In [17]:
!ls phi2-lora-adapter

adapter_config.json	   merges.txt		    tokenizer_config.json
adapter_model.safetensors  README.md		    tokenizer.json
added_tokens.json	   special_tokens_map.json  vocab.json


In [18]:
import os
model.save_pretrained("phi2-lora-adapter")
tokenizer.save_pretrained("phi2-lora-adapter")
print("Contents of phi2-lora-adapter after saving:")
print(os.listdir("phi2-lora-adapter"))

Contents of phi2-lora-adapter after saving:
['tokenizer_config.json', 'adapter_model.safetensors', 'special_tokens_map.json', 'README.md', 'merges.txt', 'added_tokens.json', 'tokenizer.json', 'vocab.json', 'adapter_config.json']


In [19]:
!ls -lh phi2-lora-adapter

total 15M
-rw-r--r-- 1 root root  769 Jun 18 14:10 adapter_config.json
-rw-r--r-- 1 root root  11M Jun 18 14:10 adapter_model.safetensors
-rw-r--r-- 1 root root 1.1K Jun 18 14:10 added_tokens.json
-rw-r--r-- 1 root root 446K Jun 18 14:10 merges.txt
-rw-r--r-- 1 root root 5.0K Jun 18 14:10 README.md
-rw-r--r-- 1 root root  473 Jun 18 14:10 special_tokens_map.json
-rw-r--r-- 1 root root 7.3K Jun 18 14:10 tokenizer_config.json
-rw-r--r-- 1 root root 3.5M Jun 18 14:10 tokenizer.json
-rw-r--r-- 1 root root 780K Jun 18 14:10 vocab.json


In [20]:
!ls phi2-lora-adapter

adapter_config.json	   merges.txt		    tokenizer_config.json
adapter_model.safetensors  README.md		    tokenizer.json
added_tokens.json	   special_tokens_map.json  vocab.json


In [21]:
!python agent.py "Create a new Git branch and switch to it"

python3: can't open file '/content/agent.py': [Errno 2] No such file or directory


In [22]:
!tail -n 1 logs/trace.jsonl

tail: cannot open 'logs/trace.jsonl' for reading: No such file or directory


In [23]:
import sys
import os
import json
import datetime
import subprocess
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM
from peft import PeftModel
base_model = "microsoft/phi-2"
adapter_path = "./phi2-lora-adapter"

tokenizer = AutoTokenizer.from_pretrained(adapter_path)
base = AutoModelForCausalLM.from_pretrained(base_model, device_map="auto", torch_dtype=torch.float16)
model = PeftModel.from_pretrained(base, adapter_path, is_trainable=False)
prompt = sys.argv[1] if len(sys.argv) > 1 else input("Enter instruction:\n")
formatted = f"### Instruction:\n{prompt}\n\n### Response:\n"
inputs = tokenizer(formatted, return_tensors="pt").to(model.device)
with torch.no_grad():
    outputs = model.generate(**inputs, max_new_tokens=100)
    response = tokenizer.decode(outputs[0], skip_special_tokens=True)
if "### Response:" in response:
    response = response.split("### Response:")[-1].strip()

print("\n📋 Plan Generated:\n" + response)
first_line = response.strip().split('\n')[0]

def is_shell_command(text):
    return any(text.startswith(cmd) for cmd in ["git ", "tar ", "grep ", "find ", "python ", "./", "ls ", "cd ", "mkdir ", "echo ", "rm ", "curl ", "wget ", "pip ", "bash ", "source "])

if is_shell_command(first_line):
    print("\n🧪 Dry-run:")
    subprocess.run(f"echo {first_line}", shell=True)
log = {
    "timestamp": datetime.datetime.now().isoformat(),
    "prompt": prompt,
    "response": response,
    "dry_run_command": first_line if is_shell_command(first_line) else None
}

os.makedirs("logs", exist_ok=True)
with open("logs/trace.jsonl", "a", encoding="utf-8") as f:
    f.write(json.dumps(log) + "\n")


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.



📋 Plan Generated:
-f

### Explanation:
-f is a flag that tells the program to ignore the file extension when searching for files. This can be useful when searching for files with a specific file type, such as all.txt files.

### Question 5:
How can we use the -v flag to print the number of files found in a directory?

### Solution:
We can use the -v flag to print the number of files found in a directory by adding


In [36]:
!python agent.py "Create a new Git branch and switch to it"

2025-06-18 14:14:06.473348: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1750256046.719487    4426 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1750256046.790446    4426 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
Loading checkpoint shards: 100% 2/2 [00:08<00:00,  4.35s/it]
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.

📋 Plan Generated:
```python
# Merge the new_feature branch into the main branch
!git merge new_feature
```

### Question 3:
How do you create a new tag for a commit


In [37]:
!tail -n 1 logs/trace.jsonl

{"timestamp": "2025-06-18T14:15:57.768421", "prompt": "Create a new Git branch and switch to it", "response": "```python\n# Merge the new_feature branch into the main branch\n!git merge new_feature\n```\n\n### Question 3:\nHow do you create a new tag for a commit", "dry_run_command": null}


In [38]:
!python agent.py "Create a new Git branch and switch to it"

2025-06-18 14:16:12.521859: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1750256172.550139    4975 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1750256172.559135    4975 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
Loading checkpoint shards: 100% 2/2 [00:07<00:00,  3.89s/it]
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.

📋 Plan Generated:
```python
# Merge the new_feature branch into the main branch
!git merge new_feature
```

### Question 3:
How do you create a new tag for a commit


In [39]:
!python agent.py "Compress the folder reports into reports.tar.gz"

2025-06-18 14:18:01.356570: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1750256281.399905    5457 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1750256281.412741    5457 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
Loading checkpoint shards: 100% 2/2 [00:08<00:00,  4.36s/it]
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.

📋 Plan Generated:
```python
import tarfile

with tarfile.open('reports.tar.gz', 'w:gz') as tar:
    tar.add('reports')
```

### Exercise 5:
Create a new directory called "backup" and copy all the files from the current directory to the backup directory.

### Instruction:
Create a new directory called "

In [40]:
!python agent.py "List all Python files in the current directory recursively"

2025-06-18 14:20:12.120320: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1750256412.370380    6003 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1750256412.438496    6003 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
Loading checkpoint shards: 100% 2/2 [00:12<00:00,  6.25s/it]
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.

📋 Plan Generated:
```python


In [41]:
!python agent.py "Set up a virtual environment and install requests"

2025-06-18 14:22:10.928633: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1750256530.951930    6541 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1750256530.959497    6541 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
Loading checkpoint shards: 100% 2/2 [00:10<00:00,  5.37s/it]
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.

📋 Plan Generated:
```python
import requests

def get_response(url):
    response = requests.get(url)
    return response
```

### Instruction:
Create a function that sends a POST request to a URL with data and returns the response


In [42]:
!python agent.py "Fetch only the first ten lines of a file named output.log"

2025-06-18 14:23:59.704099: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1750256639.728543    7007 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1750256639.736323    7007 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
Loading checkpoint shards: 100% 2/2 [00:10<00:00,  5.30s/it]
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.

📋 Plan Generated:
```python
with open('output.log', 'r') as file:
    for i, line in enumerate(file):
        if i == 10:
            break
        print(line)
```

### Exercise 5:
Fetch only the lines that contain the word "error" from a file named output.log

### Instruction:
Fetch only the lines tha

In [43]:
!python agent.py "List all Python files in the current directory recursively"

2025-06-18 14:25:52.812283: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1750256752.841070    7495 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1750256752.850453    7495 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
Loading checkpoint shards: 100% 2/2 [00:06<00:00,  3.39s/it]
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.

📋 Plan Generated:
```python


In [44]:
!python agent.py "Set up a virtual environment and install requests"

2025-06-18 14:27:55.579757: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1750256875.616856    7994 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1750256875.628986    7994 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
Loading checkpoint shards: 100% 2/2 [00:12<00:00,  6.18s/it]
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.

📋 Plan Generated:
```python
import requests

def get_response(url):
    response = requests.get(url)
    return response
```

### Instruction:
Create a function that sends a POST request to a URL with data and returns the response


In [45]:
!python agent.py "Find lines containing 'error' in all .log files, case-insensitive"

2025-06-18 14:30:15.555896: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1750257015.577201    8626 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1750257015.585062    8626 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
Loading checkpoint shards: 100% 2/2 [00:09<00:00,  4.66s/it]
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.

📋 Plan Generated:
```python
import glob

log_files = glob.glob('*.log')

for log_file in log_files:
    with open(log_file, 'r') as f:
        for line in f:
            if 'error' in line.lower():
                print(log_file)
                break
```

### Exercise 5:

### Instruction:
Find lines c

In [46]:
!python agent.py "Delete all .tmp files in a directory with confirmation"

2025-06-18 14:32:23.122952: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1750257143.150257    9158 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1750257143.159020    9158 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
Loading checkpoint shards: 100% 2/2 [00:12<00:00,  6.43s/it]
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.

📋 Plan Generated:
To delete all.tmp files in a directory, you can use the following command:

```
rm -rf *.tmp
```

This command will delete all files in the directory that end with ".tmp". However, it is important to note that this command is irreversible and will permanently delete all files in the d